In [29]:
import os
import sys
import glob
import shutil
from copy import deepcopy
from ipywidgets import interact
import matplotlib.pyplot as plt
import numpy as np

sys.path.append('..')
from preprocessing import CTImagesBatch
from dataset import FilesIndex
print('OK')

OK


### Global index

In [2]:
DIR_TEST = '/notebooks/data/MRT/nci/*'
DIR_DUMP = '/notebooks/Koryagin/Batch_dev/dir_for_dumps/'

In [3]:
if os.path.exists(DIR_DUMP):
    shutil.rmtree(DIR_DUMP)

In [4]:
ind = FilesIndex(path=DIR_TEST, dirs=True)

### Create batch by subindexing

In [5]:
batch = CTImagesBatch(ind.create_subset(ind.index[[2,5,7]]))

In [6]:
batch.indices

array(['274a81c75d244187247789bd71de2b3a',
       '645e7f46eb9b834153ecf8e2b2921fe5',
       'd73264d2c4f71ef6bf8593ac55565d93'],
      dtype='<U56')

### Test actions

* check load

In [7]:
batch.load(fmt='dicom')

In [8]:
batch_copy = deepcopy(batch)

* check attrs

In [9]:
batch.shape

array([[184, 512, 512],
       [130, 512, 512],
       [157, 512, 512]])

In [10]:
batch._bounds

array([  0, 184, 314, 471])

In [11]:
batch.upper_bounds

array([184, 314, 471])

* check resize, segment, dump

In [12]:
# note that args-order is z y x, just like _data is stores in batch
batch = batch.resize(shape=(128, 256, 256)).segment()

In [13]:
batch._bounds

array([  0, 128, 256, 384])

* check dump

In [14]:
batch.dump(dst=DIR_DUMP)

selector_events.py[LINE:53]#DEBUG    [2017-07-11 14:22:22,638]  Using selector: EpollSelector


### Build batch from dump and compare

In [15]:
ind_dumped = FilesIndex(path=DIR_DUMP + '/*', dirs=True)

In [17]:
batch_from_dump = CTImagesBatch(ind_dumped)

In [18]:
batch_from_dump.load(fmt='blosc')

In [22]:
def plot_arr_slices(height, *arrays, clim=(-1200, 300)):
    fig, axes = plt.subplots(1, len(arrays), figsize=(14, len(arrays)*8))
    
    for arr, i in zip(arrays, range(len(arrays))):
        depth = arr.shape[0]
        n_slice = int(depth * height)
        
        kwargs = dict()
        if np.max(arr) - np.min(arr) > 2.0:
            kwargs.update(clim=clim)
        else:
            kwargs.update(clim=(0, 1))
    
        axes[i].imshow(arr[n_slice], cmap=plt.cm.gray, **kwargs)
    plt.show()

In [26]:
n_pat = 0
ind_npat = batch.indices[n_pat]

In [30]:
interact(lambda height: plot_arr_slices(height, 
                                        batch.get(ind_npat, 'images'),
                                        batch_copy.get(ind_npat, 'images'),
                                        batch_from_dump.get(ind_npat, 'images')),
         height=(0.01, 0.99, 0.01))

<function __main__.<lambda>>

### XIP

In [32]:
xip = batch.make_xip()

In [34]:
interact(lambda height: plot_arr_slices(height, 
                                        batch.get(ind_npat, 'images'),
                                        xip.get(ind_npat, 'images')),
         height=(0.01, 0.99, 0.01))

<function __main__.<lambda>>